In [1]:
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle
import numpy as np
from keras import backend as K
from keras.datasets import mnist
from keras.layers import Lambda , Input ,Dense
from keras.losses import binary_crossentropy
from keras.models import Model 
import pickle

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [9]:
from keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten
from keras.models import Sequential
from keras.initializers import glorot_uniform




model = Sequential()
img_dim = (28, 28)  # Image dimension for MNIST dataset

init = glorot_uniform(seed=42)
# Assuming you have defined `latent_dim`, `init`, and `img_dim` elsewhere
latent_dim = 100  # You can choose an appropriate dimension for your latent space

# Create a Sequential model for the generator
generator = Sequential()

# Input layer and hidden layer 1
generator.add(Dense(128, input_shape=(latent_dim,), kernel_initializer=init))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))

# Hidden layer 2
generator.add(Dense(256))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))

# Hidden layer 3
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))

# Output layer
generator.add(Dense(np.prod(img_dim), activation='tanh'))
generator.add(Reshape(img_dim))

# The generator model is now complete


In [12]:
from keras.layers import Dense, LeakyReLU, Flatten, Input

# Define the image dimensions for MNIST dataset
img_dim = (28, 28)

# Define the weight initializer (e.g., Glorot/Xavier initializer)
from keras.initializers import glorot_uniform
init = glorot_uniform(seed=42)

# Create a Sequential model for the discriminator
discriminator = Sequential()

# Input layer (Flatten the image to a vector)
discriminator.add(Flatten(input_shape=(img_dim[0], img_dim[1])))

# Hidden layer 1
discriminator.add(Dense(128, kernel_initializer=init))
discriminator.add(LeakyReLU(alpha=0.2))

# Hidden layer 2
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=0.2))

# Hidden layer 3
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(alpha=0.2))

# Output layer
discriminator.add(Dense(1, activation='sigmoid'))

# The discriminator model is now complete


In [25]:
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, Flatten, Reshape
from keras.optimizers import Adam
import numpy as np

# Define the image dimensions for MNIST dataset
img_dim = (28, 28)

# Define the weight initializer (e.g., Glorot/Xavier initializer)
from keras.initializers import glorot_uniform
init = glorot_uniform(seed=42)

# Update the generator model to accept the correct input shape
generator = Sequential()

# Input layer and hidden layer 1
generator.add(Dense(128, input_shape=(latent_dim,), kernel_initializer=init))
generator.add(LeakyReLU(alpha=0.2))

# Hidden layer 2
generator.add(Dense(256))
generator.add(LeakyReLU(alpha=0.2))

# Hidden layer 3
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))

# Output layer
generator.add(Dense(np.prod(img_dim), activation='tanh'))
generator.add(Reshape(img_dim))


# The generator model summary
generator.summary()

# Create a Sequential model for the discriminator
discriminator = Sequential()

# Input layer (Flatten the image to a vector)
discriminator.add(Flatten(input_shape=(img_dim[0], img_dim[1])))

# Hidden layer 1
discriminator.add(Dense(128, kernel_initializer=init))
discriminator.add(LeakyReLU(alpha=0.2))

# Hidden layer 2
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=0.2))

# Hidden layer 3
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(alpha=0.2))

# Output layer
discriminator.add(Dense(1, activation='sigmoid'))

# The discriminator model summary
discriminator.summary()

# Define the optimizer
optimizer = Adam(lr=0.0002, beta_1=0.5)

# Compile the discriminator
discriminator.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
discriminator.trainable = False

# Create a Sequential model for the GAN
d_g = Sequential()
d_g.add(generator)
d_g.add(discriminator)

# Compile the GAN model
d_g.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])

# The GAN model summary
d_g.summary()


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 128)               16512     
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 128)               0         
                                                                 
 dense_26 (Dense)            (None, 256)               33024     
                                                                 
 leaky_re_lu_19 (LeakyReLU)  (None, 256)               0         
                                                                 
 dense_27 (Dense)            (None, 512)               131584    
                                                                 
 leaky_re_lu_20 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_28 (Dense)            (None, 784)             

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_17 (Sequential)  (None, 28, 28)            583312    
                                                                 
 sequential_18 (Sequential)  (None, 1)                 265601    
                                                                 
Total params: 848913 (3.24 MB)
Trainable params: 583312 (2.23 MB)
Non-trainable params: 265601 (1.01 MB)
_________________________________________________________________


In [26]:
import numpy as np

# Randomly set tuning parameters
epochs = 150
num_batches = 15
batch_size = 8
latent_dim = 128
smooth = 0.95
save_interval = 20  # Save generated images every 20 epochs

# Print the randomly selected tuning parameters
print(f"Randomly selected tuning parameters:")
print(f"Epochs: {epochs}")
print(f"Number of batches: {num_batches}")
print(f"Batch size: {batch_size}")
print(f"Latent dimension: {latent_dim}")
print(f"Smooth factor: {smooth}")
print(f"Save interval: {save_interval}")
for epoch in range(epochs):  # Define and loop over epochs
    for i in range(num_batches):
        # Train Discriminator weights
        discriminator.trainable = True
        
        # Real samples
        X_batch = X_train[i * batch_size:(i + 1) * batch_size]
        d_loss_real = discriminator.train_on_batch(x=X_batch, y=real * (1 - smooth))
        
        # Fake Samples
        z = np.random.normal(loc=0, scale=1, size=(batch_size, latent_dim))
        X_fake = generator.predict_on_batch(z)
        d_loss_fake = discriminator.train_on_batch(x=X_fake, y=fake)
        
        # Discriminator loss
        d_loss_batch = 0.5 * (d_loss_real[0] + d_loss_fake[0])
        
        # Train Generator weights
        discriminator.trainable = False
        d_g_loss_batch = d_g.train_on_batch(x=z, y=real)
        
        # Optionally, print or log the loss values for monitoring
        print(f"Epoch: {epoch}, Batch: {i}, D Loss: {d_loss_batch}, G Loss: {d_g_loss_batch}")

    # Optionally, you can generate and save some sample images at specific intervals
    if epoch % save_interval == 0:
        generated_images = generator.predict(random_latent_vectors)
        save_generated_images(generated_images, epoch)


Randomly selected tuning parameters:
Epochs: 150
Number of batches: 15
Batch size: 8
Latent dimension: 128
Smooth factor: 0.95
Save interval: 20


NameError: name 'fake' is not defined

In [30]:
import matplotlib.pyplot as plt
import numpy as np

# Define the number of epochs and other training parameters
epochs = 100
num_batches = 10
batch_size = 5
latent_dim = 128
smooth = 0.95
save_interval = 20  # Save generated images every 20 epochs
real = np.ones((batch_size, 1))

# Lists to store loss values for plotting
d_loss_history = []
d_g_loss_history = []

# Define placeholder functions for training the discriminator and generator
def train_discriminator():
    # Generate real and fake data
    real_data = np.random.rand(batch_size, *img_dim)  # Replace with real data
    fake_data = generator.predict(np.random.rand(batch_size, latent_dim))

    # Compute discriminator loss
    real_labels = np.ones((batch_size, 1)) * smooth
    fake_labels = np.zeros((batch_size, 1))
    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)

    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    return d_loss

def train_generator():
    # Generate fake data and calculate the generator loss
    fake_data = np.random.rand(batch_size, latent_dim)  # Use noise vectors as input
    valid_labels = np.ones((batch_size, 1))
    g_loss = d_g.train_on_batch(fake_data, valid_labels)
    return g_loss

# Training loop
for epoch in range(epochs):
    for i in range(num_batches):
        # Train the discriminator
        d_loss = train_discriminator()

        # Train the generator
        g_loss = train_generator()

    # Append the loss values to the history lists
    d_loss_history.append(d_loss[0])
    d_g_loss_history.append(g_loss[0])

    # Optionally, print or log the loss values for monitoring
    print(f"Epoch: {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss[0]}")

    # Optionally, you can generate and save some sample images at specific intervals
    if epoch % save_interval == 0:
        generated_images = generator.predict(np.random.rand(10, latent_dim))
        save_generated_images(generated_images, epoch)

# Plot the discriminator and generator loss metrics
plt.plot(d_loss_history, label='Discriminator Loss')
plt.plot(d_g_loss_history, label='Generator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Discriminator and Generator Loss')
plt.show()


1/1 [==============================] - 0s 43ms/step


KeyError: in user code:

    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py", line 1130, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\optimizers\optimizer.py", line 544, in minimize
        self.apply_gradients(grads_and_vars)
    File "C:\Users\asus\anaconda3\envs\deeplearni

In [33]:
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, Flatten, Reshape
from keras.optimizers import Adam
import numpy as np

# Define the image dimensions for MNIST dataset
img_dim = (28, 28)

# Define the weight initializer (e.g., Glorot/Xavier initializer)
from keras.initializers import glorot_uniform
init = glorot_uniform(seed=42)

# Create a Sequential model for the generator
generator = Sequential()

# Input layer and hidden layer 1
generator.add(Dense(128, input_shape=(latent_dim,), kernel_initializer=init))
generator.add(LeakyReLU(alpha=0.2))

# Hidden layer 2
generator.add(Dense(256))
generator.add(LeakyReLU(alpha=0.2))

# Hidden layer 3
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))

# Output layer
generator.add(Dense(np.prod(img_dim), activation='tanh'))
generator.add(Reshape(img_dim))

# The generator model summary
generator.summary()

# Define the number of epochs and other training parameters
epochs = 100
num_batches = 10
batch_size = 5
latent_dim = 128
smooth = 0.95
save_interval = 20  # Save generated images every 20 epochs

# Lists to store loss values for plotting
d_loss_history = []
d_g_loss_history = []

# Define placeholder functions for training the discriminator and generator
def train_discriminator():
    # Generate real and fake data
    real_data = np.random.rand(batch_size, *img_dim)  # Replace with real data
    fake_data = generator.predict(np.random.rand(batch_size, latent_dim))

    # Compute discriminator loss
    real_labels = np.ones((batch_size, 1)) * smooth
    fake_labels = np.zeros((batch_size, 1))
    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)

    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    return d_loss

def train_generator():
    # Generate fake data and calculate the generator loss
    noise = np.random.randn(batch_size, latent_dim)
    generated_images = generator.predict(noise)
    valid_labels = np.ones((batch_size, 1))
    g_loss = d_g.train_on_batch(noise, valid_labels)
    return g_loss



# Training loop
for epoch in range(epochs):
    for i in range(num_batches):
        # Train the discriminator
        d_loss = train_discriminator()

        # Train the generator
        g_loss = train_generator()

    # Append the loss values to the history lists
    d_loss_history.append(d_loss[0])
    d_g_loss_history.append(g_loss[0])

    # Optionally, print or log the loss values for monitoring
    print(f"Epoch: {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss[0]}")

    # Optionally, you can generate and save some sample images at specific intervals
    if epoch % save_interval == 0:
        generated_images = generator.predict(np.random.randn(10, latent_dim))
        save_generated_images(generated_images, epoch)

# Plot the discriminator and generator loss metrics
plt.plot(d_loss_history, label='Discriminator Loss')
plt.plot(d_g_loss_history, label='Generator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Discriminator and Generator Loss')
plt.show()


Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_41 (Dense)            (None, 128)               16512     
                                                                 
 leaky_re_lu_30 (LeakyReLU)  (None, 128)               0         
                                                                 
 dense_42 (Dense)            (None, 256)               33024     
                                                                 
 leaky_re_lu_31 (LeakyReLU)  (None, 256)               0         
                                                                 
 dense_43 (Dense)            (None, 512)               131584    
                                                                 
 leaky_re_lu_32 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_44 (Dense)            (None, 784)             

1/1 [==============================] - 0s 31ms/step


KeyError: in user code:

    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py", line 1130, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\optimizers\optimizer.py", line 544, in minimize
        self.apply_gradients(grads_and_vars)
    File "C:\Users\asus\anaconda3\envs\deeplearni

In [37]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Dense, LeakyReLU, Flatten, Reshape
from keras.models import Sequential
from keras.optimizers import Adam

# Load and preprocess the MNIST dataset
(train_images, _), (_, _) = mnist.load_data()
train_images = train_images / 127.5 - 1.0  # Normalize to the range [-1, 1]
train_images = train_images.reshape(train_images.shape[0], 784)

# Define the generator
def build_generator(latent_dim):
    generator = Sequential()
    generator.add(Dense(256, input_dim=latent_dim))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(784, activation='tanh'))
    generator.add(Reshape((28, 28, 1)))
    return generator

# Define the discriminator
def build_discriminator(img_shape):
    discriminator = Sequential()
    discriminator.add(Flatten(input_shape=img_shape))
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dense(1, activation='sigmoid'))
    return discriminator

# Build and compile the discriminator
img_shape = (28, 28, 1)
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build the GAN by connecting the generator and discriminator
latent_dim = 100
generator = build_generator(latent_dim)
discriminator.trainable = False
gan = Sequential([generator, discriminator])
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Training parameters
epochs = 10000
batch_size = 64
sample_interval = 1000

# Training the GAN
for epoch in range(epochs):
    # Train discriminator
    idx = np.random.randint(0, train_images.shape[0], batch_size)
    real_images = train_images[idx]
    labels_real = np.ones((batch_size, 1))
    labels_fake = np.zeros((batch_size, 1))
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_images = generator.predict(noise)
    d_loss_real = discriminator.train_on_batch(real_images, labels_real)
    d_loss_fake = discriminator.train_on_batch(fake_images, labels_fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Train generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    labels_g = np.ones((batch_size, 1))
    g_loss = gan.train_on_batch(noise, labels_g)
    
    # Print progress
    print(f"Epoch {epoch}/{epochs}, D Loss: {d_loss[0]}, D Accuracy: {100*d_loss[1]}", end=' ')
    print(f"G Loss: {g_loss[0]}")



2/2 [==============================] - 0s 6ms/step


ValueError: in user code:

    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\asus\anaconda3\envs\deeplearning\Lib\site-package